In [9]:
import os
import librosa
import numpy as np
import pandas as pd
import faiss

def feature_extraction(file_path):
    try:
        # load the audio file
        x, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
        # extract features from the audio
        mfcc = np.mean(librosa.feature.mfcc(y=x, sr=sample_rate, n_mfcc=50).T, axis=0)
        return mfcc
    except ValueError as e:
        # Gérer l'erreur ici (par exemple, imprimer un message d'erreur ou ignorer le fichier)
        print(f"Erreur lors du chargement de {file_path}: {e}")
        return None

directory = 'C:/Users/femia/Desktop/dogs/'
features = []

for audio in os.listdir(directory):
    audio_path = os.path.join(directory, audio)
    if os.path.getsize(audio_path) > 0:  # Vérifiez si le fichier n'est pas vide
        mfccs = feature_extraction(audio_path)
        if mfccs is not None:
            features.append(mfccs)


In [10]:
# Liste pour stocker les noms des animaux
names = []

# Parcourir le répertoire et ajouter les noms des fichiers à la liste
for filename in os.listdir(directory):
    if os.path.isfile(os.path.join(directory, filename)):
        names.append(filename)




In [11]:
def creatingIndexes(ensemble):
    d = ensemble.shape[1]
    index = faiss.IndexFlatL2(d)  # Utiliser l'indexation L2
    index.add(ensemble)
    return index


In [12]:
def normalization(embedding):
    normal_embedding=np.array(embedding,dtype=np.float32)
    normal_embedding/=np.linalg.norm(normal_embedding,axis=1)[:,np.newaxis]
    return normal_embedding

In [13]:
def searchinIndex(index, normal_embedding):
    D,I= index.search(normal_embedding, 10)
    r=pd.DataFrame({'distance':D[0],'index':I[0]})
    return r

In [14]:
normal_embeddings=normalization(features)
index=creatingIndexes(normal_embeddings)

In [15]:
query_embedding=feature_extraction("C:/Users/femia/Downloads/output_12_15_59.wav")
# Assurez-vous que `query_embedding` est un tableau 2D avec une seule requête
query_embedding = query_embedding.reshape(1, -1)
normal_query=normalization(query_embedding)

In [16]:
i=searchinIndex(index,normal_query)

In [17]:
print(i)

   distance  index
0  0.010535      9
1  0.010602    595
2  0.010610    594
3  0.010692    535
4  0.010894    325
5  0.010925    533
6  0.010959    582
7  0.011049    572
8  0.011058    563
9  0.011126    574


In [18]:
print("L'animal est un " + names[i['index'][0]].replace('.aif', ''))


L'animal est un Farley-1-P-1b
